GKM Analytics — Lab Data Cleaning Pipeline
------------------------------------------
Steps:
  1. Load CSV
  2. Inspect the raw data
  3. Remove duplicates
  4. Standardize categorical columns
  5. Handle missing values
  6. Flag and handle outliers
  7. Save clean dataset

In [1]:
import pandas as pd
import numpy as np

1. Load CSV

In [2]:
def load_data(filepath: str) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    print(f"\n{'='*55}")
    print(f"  LOADED: {filepath}")
    print(f"  Rows: {len(df)} | Columns: {df.shape[1]}")
    print(f"{'='*55}")
    return df

import os
os.chdir("/Users/gauravmishra/Documents/GKM Analytics")
df = load_data("lab_data_raw.csv")
df.head()    


  LOADED: lab_data_raw.csv
  Rows: 208 | Columns: 10


,experiment_id,temperature_C,pressure_bar,concentration_mM,reaction_time_hr,pH,catalyst_type,solvent,yield_pct,outcome
0,EXP-001,164.9,8.90,48.37,4.0,8.3,NaN,DMSO,71.9,1
1,EXP-002,145.9,7.67,56.03,0.5,6.7,Pt/Al2O3,DMSO,22.8,0
2,EXP-003,169.4,7.27,60.35,8.0,4.9,Ni,ethanol,99.6,1
3,EXP-004,195.7,7.32,43.98,1.0,5.5,Pt/Al2O3,water,97.5,1
4,EXP-005,143.0,4.24,53.36,0.5,5.5,Pt/Al2O3,ethanol,65.0,1


2. Inspect

In [3]:
def inspect(df):

    print("\n── Column Types ──")
    print(df.dtypes.to_string())

    print("\n── Missing Values ──")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if missing.empty:
        print("  None found.")
    else:
        for col, count in missing.items():
            print(f"  {col}: {count} missing ({count/len(df)*100:.1f}%)")

    print("\n── Duplicate Rows ──")
    print(f"  {df.duplicated().sum()} duplicate rows found")

    print("\n── Numeric Summary ──")
    print(df.describe().round(2).to_string())

    print("\n── Categorical Unique Values ──")
    cat_cols = [c for c in df.select_dtypes(include="object").columns if c != "experiment_id"]
    for col in cat_cols:
        print(f"  {col}: {sorted(df[col].dropna().unique())}")

inspect(df)


── Column Types ──
experiment_id        object
temperature_C       float64
pressure_bar        float64
concentration_mM    float64
reaction_time_hr    float64
pH                  float64
catalyst_type        object
solvent              object
yield_pct           float64
outcome               int64

── Missing Values ──
  pressure_bar: 10 missing (4.8%)
  concentration_mM: 15 missing (7.2%)
  pH: 22 missing (10.6%)
  catalyst_type: 36 missing (17.3%)
  solvent: 41 missing (19.7%)

── Duplicate Rows ──
  8 duplicate rows found

── Numeric Summary ──
       temperature_C  pressure_bar  concentration_mM  reaction_time_hr      pH  yield_pct  outcome
count         208.00        198.00            193.00            208.00  186.00     208.00   208.00
mean          153.04          5.58             52.14              7.74    6.71      50.58     0.42
std            65.32          2.66             35.73              8.25    1.36      30.06     0.50
min            71.40          1.10             12

3.  Remove Duplicates

In [4]:
def remove_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    total_rows = len(df)
    duplicate_rows = df.duplicated().sum()

    df_clean = df.drop_duplicates()
    final_rows = len(df_clean)

    print(f"Total rows: {total_rows}")
    print(f"Duplicate rows detected: {duplicate_rows}")
    print(f"Rows after removing duplicates: {final_rows}")

    return df_clean

df = remove_duplicates(df)

Total rows: 208
Duplicate rows detected: 8
Rows after removing duplicates: 200


4. Standardize Categorials

In [5]:
# Make a copy BEFORE modifying
df_original = df.copy()

def standardize_categoricals(df: pd.DataFrame) -> pd.DataFrame:
    """Lowercase + strip whitespace on string columns."""
    
    cat_cols = ["catalyst_type", "solvent"]
    
    for col in cat_cols:
        if col in df.columns:
            df[col] = df[col].str.strip().str.lower()
    
    return df


# Apply standardization
df = standardize_categoricals(df)


# Compare BEFORE vs AFTER
print("Before:", sorted(df_original["catalyst_type"].dropna().unique()))
print("After: ", sorted(df["catalyst_type"].dropna().unique()))

Before: ['Ni', 'Pd/C', 'Pt/Al2O3', 'pd/c']
After:  ['ni', 'pd/c', 'pt/al2o3']


5. Handle Missing 

In [6]:
def handle_missing(df: pd.DataFrame) -> pd.DataFrame:
    """
    Strategy:
      - Numeric columns → impute with median (robust to outliers)
      - Categorical columns → impute with mode
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    # Exclude outcome/target from imputation
    numeric_cols = [c for c in numeric_cols if c not in ["outcome"]]

    cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
    cat_cols = [c for c in cat_cols if c not in ["experiment_id"]]

    print("\n── Missing Value Imputation ──")
    for col in numeric_cols:
        n_missing = df[col].isnull().sum()
        if n_missing > 0:
            median_val = df[col].median()
            df[col] = df[col].fillna(median_val)
            print(f"  {col}: filled {n_missing} nulls with median ({median_val:.2f})")

    for col in cat_cols:
        n_missing = df[col].isnull().sum()
        if n_missing > 0:
            mode_val = df[col].mode()[0]
            df[col] = df[col].fillna(mode_val)
            print(f"  {col}: filled {n_missing} nulls with mode ('{mode_val}')")

    return df

df = handle_missing(df)


── Missing Value Imputation ──
  pressure_bar: filled 10 nulls with median (5.74)
  concentration_mM: filled 15 nulls with median (50.37)
  pH: filled 20 nulls with median (6.70)
  catalyst_type: filled 34 nulls with mode ('pd/c')
  solvent: filled 38 nulls with mode ('ethanol')


6. Flag and Handle Outliers

In [7]:
def handle_outliers(df: pd.DataFrame, z_threshold: float = 3.0) -> pd.DataFrame:
    """
    Flag outliers using Z-score. 
    Values beyond z_threshold standard deviations are capped (winsorized)
    rather than dropped — preserving row count while reducing noise.
    """
    numeric_cols = ["temperature_C", "pressure_bar", "concentration_mM",
                    "reaction_time_hr", "pH", "yield_pct"]
    numeric_cols = [c for c in numeric_cols if c in df.columns]

    print("\n── Outlier Detection (Z-score, threshold=3.0) ──")
    df["outlier_flag"] = False

    for col in numeric_cols:
        mean = df[col].mean()
        std = df[col].std()
        z_scores = (df[col] - mean) / std
        outliers = z_scores.abs() > z_threshold
        n_outliers = outliers.sum()

        if n_outliers > 0:
            # Cap to 3 std from mean
            lower = mean - z_threshold * std
            upper = mean + z_threshold * std
            df.loc[outliers, "outlier_flag"] = True
            df[col] = df[col].clip(lower=lower, upper=upper)
            print(f"  {col}: {n_outliers} outlier(s) capped to [{lower:.2f}, {upper:.2f}]")

    print(f"  Total rows flagged: {df['outlier_flag'].sum()}")
    return df

df = handle_outliers(df)


── Outlier Detection (Z-score, threshold=3.0) ──
  temperature_C: 1 outlier(s) capped to [-45.81, 351.92]
  concentration_mM: 1 outlier(s) capped to [-53.11, 157.19]
  Total rows flagged: 2


7. Save

In [8]:
def save_clean(df: pd.DataFrame, filepath: str) -> None:
    df.to_csv(filepath, index=False)
    print(f"\n── Saved clean data → {filepath}")
    print(f"   Final shape: {df.shape[0]} rows × {df.shape[1]} columns")

Main Pipeline

In [9]:
if __name__ == "__main__":
    df = load_data("lab_data_raw.csv")
    inspect(df)
    df = remove_duplicates(df)
    df = standardize_categoricals(df)
    df = handle_missing(df)
    df = handle_outliers(df)
    save_clean(df, "lab_data_clean.csv")

    print("\n✓ Pipeline complete. Ready for feature engineering + modeling.\n")


  LOADED: lab_data_raw.csv
  Rows: 208 | Columns: 10

── Column Types ──
experiment_id        object
temperature_C       float64
pressure_bar        float64
concentration_mM    float64
reaction_time_hr    float64
pH                  float64
catalyst_type        object
solvent              object
yield_pct           float64
outcome               int64

── Missing Values ──
  pressure_bar: 10 missing (4.8%)
  concentration_mM: 15 missing (7.2%)
  pH: 22 missing (10.6%)
  catalyst_type: 36 missing (17.3%)
  solvent: 41 missing (19.7%)

── Duplicate Rows ──
  8 duplicate rows found

── Numeric Summary ──
       temperature_C  pressure_bar  concentration_mM  reaction_time_hr      pH  yield_pct  outcome
count         208.00        198.00            193.00            208.00  186.00     208.00   208.00
mean          153.04          5.58             52.14              7.74    6.71      50.58     0.42
std            65.32          2.66             35.73              8.25    1.36      30.06     